[View in Colaboratory](https://colab.research.google.com/github/nijasnazar33/WebToMobileReportConverter/blob/master/WebToMobileReportConverter.ipynb)

### **Enter MainReportName in case you are uploading a drilldown, otherwise leave it blank.**

In [12]:
#MainReportName = "" #@param {type:"string"}


from google.colab import files
import os
import fileinput
import re


#Upload lgx
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

for file in os.listdir("/content"):
    if 'lgx' in file:
        filename = file
        if 'drill' in file or 'Drill' in file:
            mode = 'DrillDown'
        else:
            mode = 'Main'
            
            
section = 'local'
copying = True
prev_line = ''


if mode == 'Main':
    print('Converting Main report...')

    max_columns = 0
    row_width = 0
    inRows = False
    with open(filename, 'r') as f:
        for line in f:
            if line.find('<Rows') > -1:
                inRows = True
            elif line.find('</Rows') > -1:
                inRows = False
            if inRows and line.find('<Column') > -1:
                max_columns += 1
                
    if max_columns > 0:
        row_width = max_columns * 25 + 50
                
    
    with fileinput.FileInput(filename, inplace=True, backup='.bak') as file:
        for line in file:
            if section == 'local' and line.find('<ReportHeader') > -1:
                section = 'header'
            elif section == 'header' and line.find('<Body') > -1:
                section = 'body'
            elif section == 'body' and line.find('<ReportFooter') > -1:
                section = 'footer'

            if copying:
                if section == 'local':
                    if line.startswith('  <LocalData ID="localGetBookmarks"'):
                        copying = False
                    elif line.find('LeadInsights.css') > -1:
                        print(line.replace('LeadInsights.css', 'MobileAnalytics-v2.css'), end='')
                        print('  <IncludeScriptFile IncludedScriptFile="Fonts.js" />')
                    else:
                        print(line, end='')
                elif section == 'header':
                    if line.find('<LineBreak') > -1 and (prev_line.find('</Division>') > -1 or prev_line.find('</Input') > -1):
                        pass
                    elif line.find('<Spaces') > -1 and prev_line.find('reportTitle') > -1:
                        pass
                    elif line.startswith('    <Division ID="divExportIcons"'):
                        copying = False
                    elif line.find('</Input') > -1:
                        print(line, end='')
                        print('    <LineBreak LineCount="2" />')
                    elif line.find('<StaticDataRow FilterName="Custom Range" FilterValue="Custom" />') > -1:
                        print('<Remark>')
                        print(line, end='')
                        print('</Remark>')
                    else:
                        print(line, end='')
                elif section == 'body':
                    if line.find('<DataTable ') > -1 or line.find('<CrosstabTable ') > -1 or line.find('<DataMultiColumnList ') > -1:
                        if line.find('WidthScale="%"') > -1:
                            print(re.sub(r'Width="[0-9]*" WidthScale="%"', '',line), end='')
                        elif line.find('WidthScale="px"') > -1:
                            print(re.sub(r'Width="[0-9]*" WidthScale="px"', '',line), end='')
                        else:
                            print(line, end='')
                    elif line.find('<Rows') > -1:
                        print('<Rows CellSpacing="5" Width="'+str(row_width)+'" WidthScale="%">')
                    elif line.find('DataTableHeaderCellCenter') > -1:
                        print(line.replace('DataTableHeaderCellCenter', 'CenterAlign'), end='')
                    elif line.find('DataTableCellCenter') > -1:
                        print(line.replace('DataTableCellCenter', 'CenterAlign'), end='')
                    elif line.find('MetricCell') > -1:
                        print(line.replace('MetricCell', 'MetricOuterDiv'), end='')
                    elif line.find('TopMetric') > -1:
                        print(line.replace('TopMetric', 'MetricValue'), end='')
                    elif line.find('TopMetricName') > -1:
                        print(line.replace('TopMetricName', 'MetricName'), end='')
                    elif line.find('SmallText') > -1:
                        print(line.replace('SmallText', 'ThemeTextSmall'), end='')
                    elif line.find('run.leadsquared') > -1:
                        print(line.replace('https://run.leadsquared.com/LeadManagement', 'lsq:/'), end='')
                    elif line.find('@Session.AppURL~/LeadManagement') > -1:
                        print(line.replace('@Session.AppURL~/LeadManagement', 'lsq:/'), end='')
                    elif line.find('<Target Type="Report"') > -1:
                        print(line.replace('FrameID="NewWindow"', ''), end='')
                    elif line.find('<ExcelColumnFormat') > -1:
                        pass
                    else:
                        print(line, end='')                
                else:
                    print(line, end='')
            elif line.startswith('  </LocalData>'):
                copying = True
            elif line.startswith('    <IncludeHtmlFile IncludedHtmlFile="ReportTitleDecoration.htm"'):
                print(line, end='')
                copying = True
            prev_line = line
elif mode == 'DrillDown':
    aggCount = ''
    reportTitle = ''
    inSummaryHeader = False
    print('Converting Drilldown report...')
    with open(filename, 'r') as f:
        for line in f:
            if line.find('<Label ID="reportTitle"') > -1:
                reportTitle = line[line.find('Caption')+9:-1]
                reportTitle = reportTitle[0:reportTitle.find('"')]
            elif line.find('<HeaderRow') > -1:
                inSummaryHeader = True
            elif line.find('</HeaderRow') > -1:
                inSummaryHeader = False

            if inSummaryHeader and line.find('@Data.') > -1:
                aggCount = line[line.find('@Data.'):line.find('~')+1]


    with fileinput.FileInput(filename, inplace=True, backup='.bak') as file:
        for line in file:
            if section == 'local' and line.find('<ReportHeader') > -1:
                section = 'header'
            elif section == 'header' and line.find('<Body') > -1:
                section = 'body'
            elif section == 'body' and line.find('<ReportFooter') > -1:
                section = 'footer'

            if copying:
                if section == 'local':
                    if line.find('LeadInsights.css') > -1:
                        print(line.replace('LeadInsights.css', 'MobileAnalytics-v2.css'), end='')
                        print('  <IncludeScriptFile IncludedScriptFile="Fonts.js" />')
                    else:
                        print(line, end='')
                elif section == 'header':
                    if line.find('<Label ID="reportTitle"') > -1:
                        print('<Label ID="reportTitle" Caption="'+reportTitle+' ('+aggCount+')" Class="ReportTitle" />')
                        print('<IncludeHtmlFile IncludedHtmlFile="ReportTitleDecoration.htm" />')
                        print('<LineBreak LineCount="1" />')
                        print('<Division ID="getLinkToMainReport">')
                        print('<Label Class="RIGHT" Caption="&lt;&lt;back to report">')
                        print('<Action Type="Report">')
                        print('<Target Type="Report" Report="'+MainReportName+'&amp;AuthToken=@Request.AuthToken~" RequestForwarding="True">')
                        print('<WaitPage />')
                        print('</Target>')
                        print('</Action>')
                        print('</Label>')
                        print('</Division>')
                        print('<LineBreak LineCount="1" />')
                    elif line.find('ReportHeader') > -1:
                        print(line, end='')
                    else:
                        pass
                elif section == 'body':
                    if prev_line.find('<Body') > -1:
                        print('<LineBreak LineCount="1" />')
                    if line.find('<DataTable ') > -1 or line.find('<CrosstabTable ') > -1 or line.find('<DataMultiColumnList ') > -1 or line.find('<Rows') > -1:
                        if line.find('WidthScale="%"') > -1:
                            print(re.sub(r'Width="[0-9]*" WidthScale="%"', '',line), end='')
                        elif line.find('WidthScale="px"') > -1:
                            print(re.sub(r'Width="[0-9]*" WidthScale="px"', '',line), end='')
                        else:
                            print(line, end='')
                    elif line.find('DataTableHeaderCellCenter') > -1:
                        print(line.replace('DataTableHeaderCellCenter', 'CenterAlign'), end='')
                    elif line.find('DataTableCellCenter') > -1:
                        print(line.replace('DataTableCellCenter', 'CenterAlign'), end='')
                    elif line.find('run.leadsquared') > -1:
                        print(line.replace('https://run.leadsquared.com/LeadManagement', 'lsq:/'), end='')
                    elif line.find('@Session.AppURL~/LeadManagement') > -1:
                        print(line.replace('@Session.AppURL~/LeadManagement', 'lsq:/'), end='')
                    elif line.find('<HeaderRow') > -1:
                        copying = False
                    elif line.find('<ExcelColumnFormat') > -1:
                        pass
                    else:
                        print(line, end='')                
                else:
                    print(line, end='')
            elif line.find('</HeaderRow') > -1:
                copying = True

            prev_line = line
            

print('Convertion complete.')        

# Download converted file
print('Dowloading file...')
files.download(filename)


#Clear directory
for file in os.listdir("/content"):
    if 'lgx' in file:
        os.remove(file)


Saving Custom.UniversalSombo.RMNewRenewalAnalysisReport.lgx to Custom.UniversalSombo.RMNewRenewalAnalysisReport.lgx
User uploaded file "Custom.UniversalSombo.RMNewRenewalAnalysisReport.lgx" with length 65593 bytes
Converting Main report...
Convertion complete.
Dowloading file...
